In [25]:
import csv
import re
import sumy
courses = {}
course_summaries = {}
#Extract prerequisite information from the .tsv to identify {foundational, applied, meta} weights.

In [26]:
foundational = []
applied = []
meta = []

with open('courses.tsv', 'r') as csvfile:
    cols = csv.reader(csvfile, delimiter='\t')
    #loop through every class
    for col in cols:
        prereqs = col[3].split(",")

        # Determine Relevance
        if col[8]:
            relevance = col[8]
            rel_lst = foundational
        elif col[9]:
            relevance = col[9]
            rel_lst = applied
        else:
            relevance = col[10]
            rel_lst = meta
        
        #rel_list is the same list of classes but with more concise info
        course_name = col[0] + " " + col[1]
        rel_lst.append(course_name)
        
        #loop thorugh each class's pre-req
        for prereq in prereqs:
            #courses has non-repeated pre-reqs, but possible repeated follow-up classes
            if prereq not in courses:
                courses[prereq] = [course_name]
            elif course_name not in courses[prereq]:
                courses[prereq].append(course_name)
        #course summaries are course descriptions of each course 
        course_summaries[course_name] = col[4]


In [30]:
dict(list(courses.items())[0:5])

{' CS 70': ['Computer Science (COMPSCI) 194-16',
  'Computer Science (COMPSCI) 189',
  'Electrical Engineering (EL ENG) 144'],
 ' MATH 10A - MATH 10B or STAT 2': ['Psychology (PSYCH) 10',
  'Psychology (PSYCH) 101'],
 'MASSCOM 10': ['Mass Communications (MASSCOM) 130',
  'Media Studies (MEDIAST) 130'],
 'MATH 1A': ['Computer Science (COMPSCI) 61A',
  'Statistics (STAT) 134',
  'Statistics (STAT) 20',
  'Earth and Planetary Science (EPS) 109',
  'Physics (PHYSICS) 77',
  'Cognitive Science (COG SCI) 131'],
 'PHILOS 4': ['Philosophy (PHILOS) 100']}

""


In [60]:
# Remove Duplicates
remove_courses = []
for prereq in courses:
    for prereq_string in courses:
        if prereq_string != prereq and prereq in prereq_string:
            for ele in courses[prereq_string]:
                if ele not in courses[prereq]:
                    courses[prereq].append(ele)
            remove_courses.append(prereq_string)

for course in remove_courses:
    if course in courses:
        courses.pop(course)
    if course in course_summaries:
        course_summaries.pop(course)

In [31]:
dict(list(courses.items())[0:5])

{' CS 70': ['Computer Science (COMPSCI) 194-16',
  'Computer Science (COMPSCI) 189',
  'Electrical Engineering (EL ENG) 144'],
 ' MATH 10A - MATH 10B or STAT 2': ['Psychology (PSYCH) 10',
  'Psychology (PSYCH) 101'],
 'MASSCOM 10': ['Mass Communications (MASSCOM) 130',
  'Media Studies (MEDIAST) 130'],
 'MATH 1A': ['Computer Science (COMPSCI) 61A',
  'Statistics (STAT) 134',
  'Statistics (STAT) 20',
  'Earth and Planetary Science (EPS) 109',
  'Physics (PHYSICS) 77',
  'Cognitive Science (COG SCI) 131'],
 'PHILOS 4': ['Philosophy (PHILOS) 100']}

In [ ]:
with open('prereq_dictionary.txt', "w") as writefile:
    for key in sorted(courses.keys()):
        writefile.write("\n" + str(key).strip(" ") + ":\n")
        for value in courses[key]:
            writefile.write("\t" + str(value) + "\n")


In [10]:
with open("course_summaries.txt", "w") as summaries:
    for key in sorted(course_summaries.keys()):
        summaries.write("\n" + str(key).strip(" ") + ":\n")
        summaries.write("\t" + str(course_summaries[key]) + "\n")

In [ ]:
with open("relevance.txt", "w") as relevance_file:
    relevance_file.write("Foundational\n")
    relevance_file.write(str(foundational) + "\n\n")
    relevance_file.write("Applied\n")
    relevance_file.write(str(applied) + "\n\n")
    relevance_file.write("Meta\n")
    relevance_file.write(str(meta) + "\n\n")

In [24]:
import csv
import sumy
from nltk import ngrams

In [56]:
clusters = {}
cluster_descriptions = {}
courses = {}
foundational = []
applied = []
meta = []
with open('single_clustered_courses.txt', 'r') as readfile:
    
        name = col[0] + " " + col[1]
        description = col[6]
        cluster = col[7].replace("/", " & ")
        if cluster == "nan":
            continue
        if cluster not in clusters:
            clusters[cluster] = [name]
            cluster_descriptions[cluster] = [description]
        else:
            clusters[cluster].append(name)
            cluster_descriptions[cluster].append(description)


with open('./clusters/all', "w") as writeall:
    for cluster_name in clusters:
        with open('./clusters/' + cluster_name, "w") as writefile:

            writefile.write(cluster_name + "\n")
            writeall.write(cluster_name + "\n")
            for course in clusters[cluster_name]:
                writefile.write("\t" + course + "\n")
                writeall.write("\t" + course + "\n")

        with open("./tokenized_phrases/" + cluster_name, "w") as write_description_file:
            all_tokenized_phrases = []
            for description in cluster_descriptions[cluster_name]:
                all_tokenized_phrases.extend([x for x in ngrams(description.split(), 1)])
                all_tokenized_phrases.extend([x for x in ngrams(description.split(), 2)])
                all_tokenized_phrases.extend([x for x in ngrams(description.split(), 3)])
            write_description_file.write(str(all_tokenized_phrases))

In [65]:
test = clusters['Mathematics & Statistics']

In [66]:
test

['Mathematics (MATH) 191',
 'Mathematics (MATH) 54',
 'Mathematics (MATH) 170',
 'Mathematics (MATH) 128A',
 'Public Health (PB HLTH) 145',
 'Statistics (STAT) 133',
 'Statistics (STAT) 134',
 'Statistics (STAT) 135',
 'Statistics (STAT) 89A',
 'Statistics (STAT) 131A',
 'Statistics (STAT) 151A',
 'Statistics (STAT) 154',
 'Statistics (STAT) 88',
 'Statistics (STAT) 159',
 'Statistics (STAT) 152',
 'Statistics (STAT) 158',
 'Statistics (STAT) 153']

In [68]:
def tree(root, branches = []):
    for branch in branches:
        assert is_tree(branch)
    return [root] + list(branches)

def root(tree):
    return tree[0]

def branches(tree):
    return tree[1:]

def is_tree(tree):
    if type(tree) != list or len(tree) < 1:
        return False
    for branch in branches(tree):
        if not is_tree(branch):
            return False
    return True

def is_leaf(tree):
    return not branches(tree)
    
def print_tree(t, indent = 0):
    print(' ' * indent + str(root(t)))
    for b in branches(t):
        print_tree(b, indent + 1)

In [ ]:
#judge whether a course is a followup course; if it is, we want to put it in later position of a tree, and record its 
#pre-reqs; if not,we want to put it as a leaf (which is going to be implemented in treeify function).
def follow_up_course(course, prereq_dict):
    ls = []
    for key in prereq_dict.keys():
        if course in prereq_dict[key]:
            ls.append(key)
    return ls


def treeify(lst):
    # Say we only look at 'Mathematics & Statistics' cluster, we first see what follow_up_course(course) returns. 
    # If true, we need to find all keys and further look at whether they are follow_up_course, until none of the resulting 
    # keys are follow_up courses.
    # Then we place these courses as the root of the tree (like Math 1A)
    

In [74]:
tree('MATH 1B', [tree('Mathematics (MATH) 54'), tree('Statistics (STAT) 133')])

['MATH 1B', ['Mathematics (MATH) 54'], ['Statistics (STAT) 133']]